## LINE 笔记

>author: xumayi@m.scnu.edu.cn
## Refernece
[1] [Tang J, Qu M, Wang M, et al. Line: Large-scale information network embedding[C]//Proceedings of the 24th international conference on world wide web. 2015: 1067-1077.](https://dl.acm.org/doi/abs/10.1145/2736277.2741093)

[2] [【论文笔记】LINE](https://zhuanlan.zhihu.com/p/45211925)

## Introduction
**先前的方法存在的问题：** 不够实用，时间复杂度过高，对现实中的大规模网络的embedding进行学习时开销太大，不可行。e.g.，twitter followee-follower network 在2012年时就拥有175million的活跃用户
以及近两亿的关系edges，而一些经典的图嵌入算法的time complexity至少是节点数的二次方。
尽管最近的一些研究接近了大规模网络的嵌入，但这些方法要么使用不是为网络设计的间接方法，要么缺乏为网络嵌入量身定制的明确目标函数。

**解决思路：** 提出了一种网络嵌入模型LINE，LINE能够对大规模的，任意类型的网络进行建模。同时，该模型所优化的目标函数同时保存了局部和全局网络结构。

* **一阶相似性（first-order proximitie）：** 由网络中两个节点间之间的直接关联决定，连接两个节点的边的权重越大，这两个节点的一阶相似性越高。

* **二阶相似性（second-order proximitie）：** 不是通过两个节点之间的tie strength来确定，而是通过两个节点的相同的邻居节点来确定。两个节点的共同的邻居节点越多，则这两个节点的
二阶相似性越高。

**一阶相似性反应的是网络的局部结构(local structure)，二阶相似性反应的是网络的全局结构(global structure)。**

如下图所示，vertex 6和 vertex 7之间由一条强连接的边相连，因此，vertex 6 和 7 之间有很强的的一阶相似性，所以，它们
在embedding space中应该很相的vertor representation应该很相近。同时，虽然vertex 5 和 vertex 6 之间并没有直接相
连的边，但它们有许多相同的邻居节点（vertex1 ，2 ，3 ，4），因此，vertex 5 和 6 之间有很强的二阶相似性，所以，它们
在embedding space中的vertor representation也应该很相近。

<img src = 'https://i.bmp.ovh/imgs/2021/10/b77cb3219ca8cb86.png' width='500'>

**与DeepWalk的区别**

| DeepWalk | LINE |
| :------: | :------: |
| 二阶相似性 | 可考虑一阶或二阶相似性 |
| 深度优先 | 广度优先 |
| 只针对无权图 | 既可以处理无权图，也可以处理有权图（一阶相似性考虑了权重） |

## Problme definition

**Information Network:** $G=(V,E)$， $V$是节点的集合，$E$是顶点之间的边的集合，每条边$e \in E$，$e =(u,v)$，且边上的权重$w_{uv}>0$，权重
代表了两个节点之间关系的强度（也就是一阶相似的强度）。如果$G$是无向图，$(u,v) \equiv (v,u)$且$w_{uv} \equiv w_{vu}$。
如果$G$是有向图，$(u,v) \equiv (v,u)$且$w_{uv} \not\equiv w_{vu}$。

**Large-scale Information Network Embedding:** 将节点$v \in V$映射到一个低维向量空间$\mathbb{R}^{d}$中，例如通过一个
函数$f_G : V \rightarrow \mathbb{R}^d $, 通常情况下$d \ll |V|$。 同时，在向量空间$\mathbb{R}_d$中，节点间的一阶相似性和二阶相似性都被保留。

## Model Description
提出了一种模型LINE，能够保留一阶相似性和二阶相似性。

### 保留一阶相似性

对一个无向边$(i,j)$，定义 $v_i$和$v_j$之间的联合概率为 $p_{1}\left(v_{i}, v_{j}\right)=\frac{1}{1+\exp \left(-\vec{u}_{i}^{T} \cdot \vec{u}_{j}\right)}$，
其中$\vec{u}_{i} \in R^{d}$是节点$v_i$的低维向量表示。$v_i$和$v_j$之间实际概率可以定义为：$\hat{p}_{1}(i, j)=\frac{w_{i j}}{W}$，其中$W=\sum_{(i, j) \in E} w_{i j}$。

要保存一阶相似性，一种非常直接的做法就是minimize如下的目标函数：$O_{1}=d\left(\hat{p}_{1}(\cdot, \cdot), p_{1}(\cdot, \cdot)\right)$

$d(\cdot,\cdot)$是两种概率分布之间的距离，本论文种用KL-divergence (KL散度)来计算它们之间的距离，因此有：$O_{1}=-\sum_{(i, j) \in E} w_{i j} \log p_{1}\left(v_{i}, v_{j}\right)$

**NOTE:** 一阶相似性仅仅适用于无向图

### 保留二阶相似性

**NOTE:** 二阶相似性对有向图和无向图都适用。

二阶相似性认为有更多的共同邻居节点的两个节点具备更高的相似性。

在这种情况下，我们可以将与一个节点直接相连的其它节点看作是该节点的"上下文"（类似于文本中某一个单词的上下文）。因此
，每个节点在网络中都扮演了两个不同的角色：节点本身和一个其它节点的特殊"上下文"。本文引入两个向量$\vec{u}_{i}$和$\vec{u}_{i}^{\prime}$，
其中$\vec{u}_{i}$表示该节点作为节点时的向量表示， $\vec{u}_{i}^{\prime}$表示该节点作为别的节点的"上下文"时的向量表示。

对每一条有向边$(i,j)$，我们定义由顶点$v_i$产生"上下文"$v_j$的概率为：

$p_{2}\left(v_{j} \mid v_{i}\right)=\frac{\exp \left(\vec{u}_{j}^{\prime T} \cdot \vec{u}_{i}\right)}{\sum_{k=1}^{|V|} \exp \left(\vec{u}_{k}^{\prime T} \cdot \vec{u}_{i}\right)}$

其中$|V|$是与顶点$v_j$直接相连的顶点的数量。

$O_{2}=\sum_{i \in V} \lambda_{i} d\left(\hat{p}_{2}\left(\cdot \mid v_{i}\right), p_{2}\left(\cdot \mid v_{i}\right)\right)$

实际的概率分布$\hat{p}_2(v_j|v_i)= \frac{w_{ij}}{d_i}$，$w_{ij}$是边$(i,j)$的权重，$d_i$是节点$i$的出度(out-drefree)，$d_i=\sum_{k \in N(i)}w_{ik}$,
$N(i)$是$v_i$的邻居节点，在该论文中，简便起见，$\lambda_i$被设定为节点$v_i$的degree，$\lambda_i =d_i$，
同时，在这里，该论文也采用KL-divergence来计算两种概率分布之间的距离，得到$O_{2}=-\sum_{(i, j) \in E} w_{i j} \log p_{2}\left(v_{j} \mid v_{i}\right)$